# modified version of an online, from-scratch tutorial on GA
tutorial used (notebook can be found at bottom of link): https://medium.com/datacat/simple-genetic-algorithm-in-python-from-scratch-d87cd88626c5

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np
import random as rnd
import math
import matplotlib.pyplot as plt
import time

In [ ]:
def distanceBTWNpts(site1, site2):
    d = math.sqrt((site2[1]-site1[1])**2 + (site2[0]-site1[0])**2)
    return d

def initialize(pop_size, decision_variables):
    pop_bag = []
    for i in range(pop_size):
        rnd_sol = decision_variables.copy()
        rnd.shuffle(rnd_sol)
        pop_bag.append(rnd_sol)
        pop_bag
    return pop_bag


def fitness_function(solution, sites):
    return_trip =distanceBTWNpts(sites[solution[len(solution)-1]], sites[solution[0]])
    solution_fitness=naive_fitness_function(solution, sites)+return_trip
    return solution_fitness

def naive_fitness_function(solution, sites):
    tot_distance = 0
    for i in range(len(solution)-1):
        tot_distance += distanceBTWNpts(sites[solution[i]], sites[solution[i+1]])
        
    return tot_distance


#--- EVALUATE FITNESS IN POPULATION BAG ---
def eval_fit_population(pop_bag, sites):
    result = {}
    result["fit_vals"] = [fitness_function(solution, sites) for solution in pop_bag]
    min_wgh = [1.01*np.max(list(result["fit_vals"]))-i for i in result["fit_vals"]]
    result["fit_wgh"]  = [i/sum(min_wgh) for i in min_wgh]
    result["solution"] = pop_bag
    return result


In [ ]:
def pickOne(fit_bag_evals):
    local_fit_bag_evals= fit_bag_evals
    a=True
    while a:
        rnIndex = rnd.randint(0, len(fit_bag_evals["solution"])-1)
        rnPick  = fit_bag_evals["fit_wgh"][rnIndex]
        r = rnd.random()
        if  r <= rnPick:
            pickedSol = fit_bag_evals["solution"][rnIndex]
            a = False
    
    return pickedSol, local_fit_bag_evals


def crossover(solA, solB):
    n     = len(solA)
    child = [np.nan for i in range(n)]
    num_els = np.ceil(n*(rnd.randint(10,90)/100))
    str_pnt = rnd.randint(0, n-1-1)
    end_pnt = n if int(str_pnt+num_els) > n else int(str_pnt+num_els)
    blockA = list(solA[str_pnt:end_pnt])
    child[str_pnt:end_pnt] = blockA
    for i in range(n):
        if list(blockA).count(solB[i]) == 0:
            for j in range(n):
                if np.isnan(child[j]):
                    child[j] = solB[i]
                    break
    return child


def mutation(sol):
    n = len(sol)
    pos_1 = rnd.randint(0,n-1)
    pos_2 = rnd.randint(0,n-1)
    result = swap(sol, pos_1, pos_2)
    return result

def swap(sol, posA, posB):
    result = sol.copy()
    elA = sol[posA]
    elB = sol[posB]
    result[posA] = elB
    result[posB] = elA
    return result

In [ ]:
def polygonVerticesadvanced(numPoints, radius, centerX, centerY):
    vertices = []
    xcoordinate = []
    ycoordinate = []
    for i in range(numPoints):
        crtAngle=math.pi/numPoints + (i) * (2*math.pi)/(numPoints)
        xcoordinate.append(radius*math.cos(crtAngle))
        ycoordinate.append(radius*math.sin(crtAngle))
    vertices = list(zip(
        [x + centerX for x in xcoordinate]
        , [x + centerY for x in ycoordinate]))
   
    return vertices



def plot_solution(sites, solution):
    x = [i[0] for i in sites] 
    y = [i[1] for i in sites]
    plt.scatter(x, y) 

    annotations=[i for i in range(len(sites))]
    for i, label in enumerate(annotations):
        plt.annotate(label, (x[i], y[i]+0.05))
    
    x = [sites[i][0] for i in solution]
    y = [sites[i][1] for i in solution]
    x.append(sites[0][0])
    y.append(sites[0][1])
    plt.plot(x,y)
    
    plt.show()




In [ ]:
POP_SIZE = 50
N_SITES = 20
N_GENERATIONS = 20
EVERY_N_GENS = 5

decision_variables = [*range(N_SITES)]

sites=polygonVerticesadvanced(N_SITES,1,0,0)
sites

init_popAA=initialize(pop_size=POP_SIZE, decision_variables= [*range(5)])
init_popAA

init_pop=initialize(POP_SIZE, decision_variables)
init_pop

init_pop[0][3:7]

fit_bag_evals=eval_fit_population(init_pop, sites)
fit_bag_evals

a, _ = pickOne(fit_bag_evals)
a

plot_solution(sites, a)    

In [ ]:
def running_everything(pop_size, n_generations, every_n_gens):
    pop_bag  = initialize(pop_size, decision_variables)
    
    fit_global_history = []
    fit_history = []
    fit_global_history_g = []
    best_fit_global=math.inf


    for g in range(n_generations):
#         print("generation "+ str(g)+" start:")


        pop_bag_fit = eval_fit_population(pop_bag, sites)


        best_fit = np.min(pop_bag_fit["fit_vals"])
        best_fit_index = pop_bag_fit["fit_vals"].index(best_fit)
        best_solution  = pop_bag_fit["solution"][best_fit_index]
#     print(pop_bag_fit)
#     print(best_fit, best_solution)
        if g %every_n_gens==0: 
            plot_solution(sites, best_solution)
#         print(popbag, all 3s)


        if (best_fit <= best_fit_global):
            best_fit_global      = best_fit
            best_solution_global = best_solution
            fit_global_history.append(best_fit_global)
            fit_global_history_g.append(g)             
                
        fit_history.append(best_fit)

        
        new_pop_bag = []

        for i in range(pop_size):
#             print("index"+ str(i)+" start:")

            pA, _  = pickOne(pop_bag_fit)
            pB, _  = pickOne(pop_bag_fit)
            new_element = pA

            if rnd.random() <= 0.87:
                new_element = crossover(pA, pB)

            if rnd.random() <= 0.7:
                new_element = mutation(new_element) 

            new_pop_bag.append(new_element)
#         print("index"+ str(i)+" end.")


        pop_bag = np.array(new_pop_bag)
    print("generation "+ str(g)+" end.")



    print(f"Best Fitness: {best_fit_global}")
    print(f"Best Solution: {best_solution_global}")
    plot_solution(sites, best_solution)
 
    print(fit_history)
    print(fit_global_history)
    return fit_history, fit_global_history, fit_global_history_g

In [ ]:
POP_SIZE = 300
N_SITES = 20
N_GENERATIONS = 50
EVERY_N_GENS = 5

start_time = time.time()
fit_history, fit_global_history, fit_global_history_g = running_everything(POP_SIZE, N_GENERATIONS, EVERY_N_GENS)
print(" %s seconds " % (time.time() - start_time))

In [ ]:
len(fit_history)
len([*range(N_GENERATIONS)])
len(fit_global_history_g)
len(fit_global_history)

true_minimum = fitness_function([*range(N_SITES)], sites)

plt.bar([*range(N_GENERATIONS)], fit_history, label = 'search_fit', alpha=0.3, color=['green']) 
plt.plot(fit_global_history_g, fit_global_history, 'b', drawstyle='steps-post', label = 'global_fit') 
plt.hlines(y=true_minimum, xmin=0, xmax=N_GENERATIONS, colors='r', linestyles='-', lw=2, label='true_optimum')
plt.legend(loc='lower left')
plt.show()